In [1]:
import pandas as pd

### Import des fichiers

In [2]:
years = ["2014","2015","2016","2017"]

In [3]:
data_immo = {}
for year in years:
    tempo = pd.read_csv('../../data/foncier/immobilier_' + year + '_clean_month.csv',sep=',',dtype= {'code_commune' : str})
    tempo.rename(columns = {'valeur_fonciere' : 'valeur' },inplace=True)
    data_immo[year] = tempo

In [4]:
data_immo['2017'].sort_values(by=['id_bien','year','month']).head()

,id_bien,month,year,surface,nb_mutation,valeur
1337280,01001_1_2,1,2017,0.0,0.0,0.0
1337281,01001_1_2,2,2017,0.0,0.0,0.0
1337282,01001_1_2,3,2017,0.0,0.0,0.0
1337283,01001_1_2,4,2017,0.0,0.0,0.0
1337284,01001_1_2,5,2017,0.0,0.0,0.0


In [5]:
data_immo['2017'].shape

(1733400, 6)

### Ne garder que les biens qui sont présents sur toutes les années

on récupère d'abord les ids de biens présent sur toutes les années

In [6]:
id_bien_all_year = {}
for key, data in data_immo.items():
    
    temp = set(data["id_bien"].drop_duplicates())
    
    if len(id_bien_all_year) == 0:
        id_bien_all_year = temp
    else:
        id_bien_all_year = id_bien_all_year.intersection(temp)
    print(len(id_bien_all_year))    


127506
85124
69998
62593


on filtre ensuite les biens en fonctions de ces ids de biens

In [7]:
for key, data in data_immo.items():
    data_immo[key] = data[data.id_bien.isin(id_bien_all_year)]

### Concaténer les data frames en un seul

In [8]:
list(data_immo.values())

[           id_bien  month  year     surface  nb_mutation         valeur
 0        93072_2_4      1  2014   70.000000          5.0  179202.804000
 1        93072_2_4      2  2014   72.666667          3.0  204337.850000
 2        93072_2_4      3  2014   74.500000          2.0  142500.000000
 3        93072_2_4      4  2014   79.000000          3.0  193708.880000
 4        93072_2_4      5  2014   68.750000          4.0  169725.465000
 5        93072_2_4      6  2014   83.000000          1.0  160000.000000
 6        93072_2_4      7  2014   73.375000          8.0  208857.476250
 7        93072_2_4      8  2014   75.500000          2.0  171500.000000
 8        93072_2_4      9  2014   77.500000          2.0  184500.000000
 9        93072_2_4     10  2014   81.200000          5.0  213588.786000
 10       93072_2_4     11  2014   72.000000          1.0  180000.000000
 11       93072_2_4     12  2014   75.500000          2.0  212500.000000
 12       93001_2_2      1  2014   41.764706       

In [9]:
immo_concat = pd.concat(list(data_immo.values()))

In [10]:
immo_concat.shape

(3004464, 6)

### Filtre des lignes selon le nombre de mutation ( on vire tous les types de biens avec moins de 10 ventes sur 2014 - 2017 )

In [37]:
immo_concat.groupby('id_bien').sum().nb_mutation.value_counts(sort=True).sort_index().head(10)

4.0     1397
5.0     2538
6.0     3222
7.0     3512
8.0     3405
9.0     3198
10.0    2922
11.0    2739
12.0    2554
13.0    2255
Name: nb_mutation, dtype: int64

On va virer toutes les types de biens qui totalisent moins de 10 mutations sur toutes les années

In [38]:
sum_by_id_bien = immo_concat.groupby('id_bien').sum().reset_index()

In [39]:
sum_by_id_bien.head()

,id_bien,month,year,surface,nb_mutation,valeur
0,01001_1_3,312,96744,399.0,5.0,9.123900e+05
1,01001_1_4,312,96744,923.0,13.0,1.757733e+06
2,01004_1_1,312,96744,428.5,11.0,8.456350e+05
3,01004_1_2,312,96744,1022.0,24.0,1.825696e+06
4,01004_1_3,312,96744,2744.5,66.0,5.612306e+06


In [40]:
sum_by_id_bien.shape

(62593, 6)

In [41]:
sum_by_id_bien = sum_by_id_bien[sum_by_id_bien.nb_mutation > 100]

In [42]:
sum_by_id_bien.shape

(4750, 6)

In [43]:
62593-42399

20194

Cela vire 20 194 biens

In [44]:
id_bien_to_keep = sum_by_id_bien.id_bien

In [45]:
immo_concat.shape

(3004464, 6)

In [46]:
immo_concat = immo_concat[immo_concat.id_bien.isin(id_bien_to_keep)]
immo_concat.shape

(228000, 6)

Cela vire 969312 lignes

In [47]:
immo_concat.head()

,id_bien,month,year,surface,nb_mutation,valeur
0,93072_2_4,1,2014,70.000000,5.0,179202.804
1,93072_2_4,2,2014,72.666667,3.0,204337.850
2,93072_2_4,3,2014,74.500000,2.0,142500.000
3,93072_2_4,4,2014,79.000000,3.0,193708.880
4,93072_2_4,5,2014,68.750000,4.0,169725.465


In [48]:
immo_concat.shape

(228000, 6)

### On va créer un identifiant de bien immobilier entier

In [49]:
id_bien_all_year

{'85163_2_1',
 '23075_1_3',
 '26166_2_4',
 '14739_1_6',
 '17299_1_1',
 '71208_1_3',
 '03023_2_2',
 '44049_2_1',
 '07220_1_3',
 '59156_1_3',
 '31481_1_3',
 '80620_2_2',
 '01241_1_3',
 '37018_1_5',
 '01224_2_3',
 '88196_1_7',
 '78172_2_4',
 '09065_1_2',
 '33293_1_3',
 '40274_1_6',
 '86273_1_5',
 '24543_1_3',
 '63301_1_7',
 '85190_1_5',
 '37062_1_2',
 '33080_1_2',
 '44131_1_2',
 '51395_1_4',
 '49218_1_6',
 '42218_1_8',
 '19278_1_5',
 '80442_1_4',
 '09238_1_6',
 '85001_1_5',
 '42187_2_1',
 '43157_1_5',
 '28309_1_3',
 '74279_2_4',
 '65262_1_3',
 '80241_1_4',
 '07010_2_3',
 '93001_1_5',
 '06153_1_3',
 '76663_1_5',
 '18019_1_2',
 '97418_1_6',
 '66175_1_4',
 '07331_1_1',
 '46127_2_1',
 '74243_1_5',
 '35238_2_1',
 '80523_1_4',
 '50031_1_6',
 '81182_1_6',
 '63178_1_4',
 '14383_1_6',
 '94033_1_1',
 '17303_1_3',
 '58300_1_3',
 '13045_1_1',
 '35015_1_5',
 '97417_1_3',
 '13213_1_4',
 '43152_1_5',
 '38082_1_4',
 '79285_1_4',
 '80533_2_4',
 '34299_1_1',
 '01043_1_5',
 '33138_1_1',
 '40117_1_4',
 '7416

In [50]:
d = {'id_bien': list(id_bien_all_year)}
bien_table = pd.DataFrame(data=d)

In [51]:
bien_table.head()

,id_bien
0,85163_2_1
1,23075_1_3
2,26166_2_4
3,14739_1_6
4,17299_1_1


In [52]:
bien_table['id_bien_int'] = bien_table.index

### On ajoute cet identifiant de bien immobilier au dataset immo

In [53]:
immo_concat = pd.merge(bien_table, immo_concat, on='id_bien', how='inner')

In [54]:
bien_table.head()

,id_bien,id_bien_int
0,85163_2_1,0
1,23075_1_3,1
2,26166_2_4,2
3,14739_1_6,3
4,17299_1_1,4


In [55]:
immo_concat.head()

,id_bien,id_bien_int,month,year,surface,nb_mutation,valeur
0,78172_2_4,16,1,2014,87.400000,5.0,244117.392000
1,78172_2_4,16,2,2014,75.333333,3.0,256866.666667
2,78172_2_4,16,3,2014,69.500000,4.0,193337.500000
3,78172_2_4,16,4,2014,74.200000,5.0,181700.000000
4,78172_2_4,16,5,2014,79.500000,4.0,184250.000000


In [56]:
immo_concat.drop(columns='id_bien',inplace=True)

### On ajoute la variable mois

In [57]:
immo_concat = immo_concat.drop(columns=["surface","nb_mutation"])

### On enregistre comme csv

In [58]:
immo_concat.to_csv('../../data/join/immo_by_month_100.csv',index=False)

In [59]:
bien_table.to_csv('../../data/table/type_bien_immo.csv',index=False)